#### Here I am calculating the initialization for the compartment, I am simulating from 20th of MAY 2020.
- the result that I got from here, will be used as initialization in the other py scripts

In [18]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from iprocessor import add_day_name_column, add_date_name_column, smooth_sundays_rolling_w7_l

In [63]:
def get_value_by_date(input_date, input_dataframe, column_name):
    """
    Get the value from the specified column in the DataFrame based on the provided date.

    Parameters:
    - input_date (str or pd.Timestamp): The date to look for in the DataFrame.
    - input_dataframe (pd.DataFrame): The DataFrame containing the data.
    - column_name (str): The column from which to extract the value.

    Returns:
    - value: The value from the specified column corresponding to the given date.
    """
    try:
        # Normalize the input date to midnight
        normalized_date = pd.to_datetime(input_date).normalize()

        # Normalize the 'Date' column in the dataframe for comparison
        input_dataframe['Date'] = pd.to_datetime(input_dataframe['Date']).dt.normalize()

        # Filter the dataframe to get the row corresponding to the date
        filtered_df = input_dataframe[input_dataframe['Date'] == normalized_date]

        # Check if the filtered dataframe is empty
        if filtered_df.empty:
            print(f"No data available for the date {normalized_date.strftime('%Y-%m-%d %H:%M:%S')} in the DataFrame.")
            return None

        # Extract the value from the specified column
        value = filtered_df[column_name].iloc[0]

        return value

    except IndexError:
        print(f"No data available for the date {normalized_date.strftime('%Y-%m-%d %H:%M:%S')} in the DataFrame.")
        return None


In [64]:
### Here we are using one of the RKI data csv

In [65]:
df_observed = pd.read_csv(r'German_case_period_may_aug.csv')

In [66]:
df_observed.describe()

,Confirmed,Deaths,Recovered,n_confirmed,n_death,n_recovered,Infection_case
count,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000
mean,190289.118280,9182.956989,181106.161290,504.752688,9.000000,495.752688,520.569892
std,12450.089224,203.979907,12258.059448,182.274140,9.258369,177.244713,192.136014
min,166468.000000,8602.000000,157866.000000,0.000000,0.000000,0.000000,186.000000
25%,180754.000000,9094.000000,171660.000000,380.000000,4.000000,377.000000,381.000000
50%,189828.000000,9242.000000,180586.000000,478.000000,6.000000,475.000000,485.000000
75%,200078.000000,9331.000000,190747.000000,631.000000,10.000000,622.000000,639.000000
max,213410.000000,9439.000000,203971.000000,915.000000,43.000000,889.000000,1066.000000


In [67]:
df_observed.dtypes

Date              object
Confirmed          int64
Deaths             int64
Recovered          int64
n_confirmed        int64
n_death            int64
n_recovered        int64
Infection_case     int64
dtype: object

In [68]:
df_observed.head(20)

,Date,Confirmed,Deaths,Recovered,n_confirmed,n_death,n_recovered,Infection_case
0,2020-05-01,166468,8602,157866,0,0,0,915
1,2020-05-02,167160,8626,158534,692,24,668,846
2,2020-05-03,167753,8650,159103,593,24,569,805
3,2020-05-04,168585,8693,159892,832,43,789,607
4,2020-05-05,169481,8731,160750,896,38,858,447
5,2020-05-06,170396,8757,161639,915,26,889,682
6,2020-05-07,171242,8793,162449,846,36,810,639
7,2020-05-08,172047,8827,163220,805,34,771,708
8,2020-05-09,172654,8834,163820,607,7,600,631
9,2020-05-10,173101,8859,164242,447,25,422,739


In [72]:
# parameters in days
total_population = 82000000
exposed_period = 5.3
asymptomatic_period = 7
infectious_period = 3.6
isolated_period = 11
prob_asymptomatic = 0.2
prob_quarant_inf = 0.05

In [73]:
# Initialize date
init_date = pd.to_datetime('2020-05-20')
df_observed['Date'] = pd.to_datetime(df['Date'])

# Check if the chosen init_date exists in the dataframe
if init_date < df['Date'].min() or init_date > df['Date'].max():
    raise ValueError('The date is out of the range of the dataframe')

# Initialization of the compartments
date_init = init_date
date_ec = init_date + pd.Timedelta(days=exposed_period + asymptomatic_period)
date_c = init_date + pd.Timedelta(days=asymptomatic_period)
date_i = init_date - pd.Timedelta(days=infectious_period)
date_if = init_date - pd.Timedelta(days=isolated_period + infectious_period)

# Ensure prob_asymptomatic is not 1 to avoid division by zero
if prob_asymptomatic == 1:
    raise ValueError("Probability of asymptomatic cases cannot be 1")

E0 = (1 / (1 - prob_asymptomatic)) * (get_value_by_date(date_ec, df_observed, 'Confirmed') - get_value_by_date(date_c, df_observed, 'Confirmed'))
A0 = (1 / (1 - prob_asymptomatic)) * (get_value_by_date(date_c, df_observed, 'Confirmed') - get_value_by_date(date_init, df_observed, 'Confirmed'))
I0 = prob_quarant_inf * (get_value_by_date(date_i, df_observed, 'Confirmed') - get_value_by_date(date_if, df_observed, 'Confirmed'))
F0 = get_value_by_date(init_date, df_observed, 'Confirmed') - get_value_by_date(date_i, df_observed, 'Confirmed')
R0 = get_value_by_date(init_date, df_observed, 'Recovered')
D0 = get_value_by_date(init_date, df_observed, 'Deaths')
S0 = total_population - E0 - A0 - I0 - R0 - D0 - F0

print(f"E0: {E0}, A0: {A0}, I0: {I0}, F0: {F0}, R0: {R0}, D0: {D0}, S0: {S0}")

E0: 2026.25, A0: 3798.75, I0: 376.40000000000003, F0: 2255, R0: 170204, D0: 9060, S0: 81812279.6
